In [1]:
pip install transformers datasets torch


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install 'accelerate>=0.26.0'


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'accelerate"


In [3]:
pip install datasets 


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade datasets


Note: you may need to restart the kernel to use updated packages.


---

### gpt2


In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 모델과 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# 모델 실행
output = model.generate(input_ids, max_length=50, num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Hello, how are you?

I'm a little bit of a nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a


---

### json 파일 로드 -> 학습


In [6]:
import pandas as pd
import os
import json
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch


In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device


device(type='cuda')

In [8]:
folder_path='./json/'

os.listdir(folder_path)


['doctissimo-11-29.json',
 'doctissimo-12-02.json',
 'lemonde-10-23.json',
 'lemonde-11-29.json',
 'lemonde-12-02.json',
 'lemonde-12-03.json',
 'lemonde-culture-12-03.json',
 'lemonde-debat-12-03.json',
 'lemonde-economie-12-03.json',
 'lemonde-gout-12-03.json']

In [9]:
files_data=[]

for file in os.listdir(folder_path):
    file_path=folder_path+file
    print(file_path)

    with open(file_path,'r',encoding='utf-8') as f:
        doc=json.load(f)

    for d in doc:
        if isinstance(d,dict) and 'content' in d:
            print(d)
            df=pd.DataFrame.from_dict([d])
            files_data.append(df)

combine_df=pd.concat(files_data,ignore_index=True)


./json/doctissimo-11-29.json
{'url': 'https://www.doctissimo.fr/medicaments/actualites-medicaments/prescrire-alerte-106-medicaments-juges-plus-risques-quutiles-en-2025-lesquels-eviter/ac0863_ar.html', 'date': '2024-11-29 20:14:57', 'content': 'Prescrire alerte, 106 médicaments jugés plus risqués qu’utiles en 2025 : lesquels éviter ?\nPour la 13e année consécutive, la revue Prescrire publie dans son numéro de décembre 2024, sa liste des "médicaments autorisés mais plus dangereux qu\'utiles" qui sont "à écarter pour mieux soigner". A vérifier dans vos prescriptions.\nPour la 13e année consécutive, la revue Prescrire publie dans son numéro de décembre 2024, sa liste des "médicaments autorisés mais plus dangereux qu\'utiles" qui sont "à écarter pour mieux soigner". A vérifier dans vos prescriptions.\nComme tous les ans depuis 13 ans, la revue Prescrire publie en décembre un bilan des médicaments autorisés mais "plus dangereux qu\'utiles". Le bilan 2025 porte sur 106 médicaments (dont 88 co

In [10]:
combine_df


,url,date,content,summary
0,https://www.doctissimo.fr/medicaments/actualit...,2024-11-29 20:14:57,"Prescrire alerte, 106 médicaments jugés plus r...","pour la 13e année consécutive, la revue prescr..."
1,https://www.doctissimo.fr/sante/sante-au-quoti...,2024-11-29 20:14:57,Les 5 conseils du Dr Gérald Kierzek pour ne pa...,"pour ne plus subir ces aléas saisonniers, adop..."
2,https://www.doctissimo.fr/sante/lessentiel-de-...,2024-11-29 20:14:58,"3 signes alarmants pour votre vision, selon un...",certains symptômes ne doivent pas être ignorés...
3,https://www.doctissimo.fr/nutrition/aliments-e...,2024-11-29 20:14:59,"Foie gras, le verdict de l'UFC-Que Choisir sur...",selon l'association ufc-que choisir un label e...
4,https://www.doctissimo.fr/nutrition/alimentati...,2024-11-29 20:15:00,"Cette huile n’est pas recommandée en cuisine, ...","tournesol, lin, coco, olive… il existe différe..."
...,...,...,...,...
676,https://www.lemonde.fr/m-styles/article/2024/1...,2024-12-03 21:50:43,"Chez Stock, à Paris, « pile, le goût noiseté d...","chez stock, à paris, « pile, le goût noiseté d..."
677,https://www.lemonde.fr/le-monde-passe-a-table/...,2024-12-03 21:50:44,"La spatule coudée, au bonheur des tartes\nSi, ...",en lui faisant faire des va-et-vient du poigne...
678,https://www.lemonde.fr/m-styles/article/2024/1...,2024-12-03 21:50:45,"Le lys de Malabar, une beauté vénéneuse\nLes f...","fragile coupé, très résistant sur pied,glorios..."
679,https://www.lemonde.fr/m-styles/article/2024/1...,2024-12-03 21:50:46,48 heures à Clermont-Ferrand\nReportageGrimper...,14 h 30. un bol d’air comprimé et si c’était m...


In [11]:
# GPT-2 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# pad_token을 eos_token으로 설정
tokenizer.pad_token = tokenizer.eos_token


In [12]:
def prepare_inputs(examples):
    # 입력 텍스트와 요약문을 결합
    inputs = [str(content) + " TL;DR: " + str(summary) for content, summary in zip(examples['content'], examples['summary'])]
    
    # 토크나이징
    tokenized = tokenizer(
        inputs, 
        padding="max_length", 
        truncation=True, 
        max_length=512,
        return_tensors="pt"
    )
    
    # 레이블도 동일한 토크나이즈된 입력으로 설정
    tokenized['labels'] = tokenized['input_ids'].clone()
    
    return tokenized


In [13]:
# Pandas DataFrame을 Hugging Face Datasets 객체로 변환
dataset = Dataset.from_pandas(combine_df[['content', 'summary']])

# 데이터셋 분할
train_test_split = dataset.train_test_split(test_size=0.2)

# 데이터셋 준비
tokenized_train_dataset = train_test_split['train'].map(
    prepare_inputs, 
    batched=True, 
    remove_columns=train_test_split['train'].column_names
)

tokenized_eval_dataset = train_test_split['test'].map(
    prepare_inputs, 
    batched=True, 
    remove_columns=train_test_split['test'].column_names
)


Map:   0%|          | 0/544 [00:00<?, ? examples/s]

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

In [ ]:
# 데이터 콜레이터 생성: 텍스트 데이터를 모델이 학습할 수 있는 형태로 변환하는 도구
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False  # GPT-2는 masked language modeling을 사용하지 않음
)


mlm=False

- mlm (Masked Language Modeling)은 마스크된 언어 모델링을 의미
- BERT 모델은 입력 문장에서 일부 단어를 마스킹하고, 그 단어를 예측하는 방식으로 학습 진행
- 그러나 GPT-2 모델은 "Autoregressive" 모델이므로 마스킹을 사용하지 않고, 주어진 문장의 이전 단어들을 기반으로 다음 단어를 예측하는 방식으로 학습 진행

In [15]:
from transformers import EarlyStoppingCallback


In [ ]:
# GPT-2 모델 로드
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',            # 출력 결과 저장 폴더
    evaluation_strategy="epoch",       # 평가 주기: 각 에포크가 끝날 때마다 평가 수행
    learning_rate=5e-5,                # 학습률
    per_device_train_batch_size=4,     # 배치 사이즈
    num_train_epochs=500,              # 에포크 수
    weight_decay=0.01,                 # 가중치 감쇠 for. 과적합 방지
    logging_dir='./logs',              # 로그 저장 폴더
    remove_unused_columns=False,       # 사용되지 않는 열(인덱스)을 제거하도록
    save_strategy="epoch",             # 체크포인트를 에포크마다 저장하도록
    load_best_model_at_end=True,       # 학습 후, 평가 성능이 가장 좋은 모델 로드
    metric_for_best_model='eval_loss', # 평가 기준: eval_loss
    save_total_limit=2,                # 체크포인트는 2개까지만 저장 (그 이후로는 이전 체크포인트 덮어씀)
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=30)]
)

# 학습 시작
trainer.train()


c:\ProgramData\miniconda3\envs\torch_nlp38\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4844\28022166.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/68000 [00:00<?, ?it/s]

c:\ProgramData\miniconda3\envs\torch_nlp38\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 3.384343385696411, 'eval_runtime': 1.6426, 'eval_samples_per_second': 83.405, 'eval_steps_per_second': 10.958, 'epoch': 1.0}


...

  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 3.5906219482421875, 'eval_runtime': 1.6139, 'eval_samples_per_second': 84.888, 'eval_steps_per_second': 11.153, 'epoch': 48.0}


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'train_runtime': 1149.6683, 'train_samples_per_second': 236.59, 'train_steps_per_second': 59.147, 'train_loss': 1.5037650099279833, 'epoch': 48.0}


TrainOutput(global_step=6528, training_loss=1.5037650099279833, metrics={'train_runtime': 1149.6683, 'train_samples_per_second': 236.59, 'train_steps_per_second': 59.147, 'total_flos': 6822857539584000.0, 'train_loss': 1.5037650099279833, 'epoch': 48.0})

In [17]:
model.save_pretrained('./summarization_model')
tokenizer.save_pretrained('./summarization_model')


('./summarization_model\\tokenizer_config.json',
 './summarization_model\\special_tokens_map.json',
 './summarization_model\\vocab.json',
 './summarization_model\\merges.txt',
 './summarization_model\\added_tokens.json')

In [31]:
text="""

Le Changement Climatique et le Développement Durable : Une Urgence Planétaire

Le changement climatique est l'un des plus grands défis auxquels l'humanité fait face au XXIe siècle. Les émissions de gaz à effet de serre, principalement dues à l'activité humaine, entraînent une hausse des températures mondiales, des phénomènes météorologiques extrêmes et une élévation du niveau des océans. Pour faire face à cette crise environnementale, le concept de développement durable est devenu essentiel.

Le développement durable repose sur l'idée qu'il est possible de répondre aux besoins des générations présentes sans compromettre la capacité des générations futures à satisfaire leurs propres besoins. Il s'agit de trouver un équilibre entre la protection de l'environnement, le progrès économique et l'amélioration des conditions sociales.

Les actions pour lutter contre le changement climatique incluent la réduction des émissions de gaz à effet de serre, la transition vers les énergies renouvelables, ainsi que la promotion de l'efficacité énergétique. Les entreprises, les gouvernements et les citoyens ont un rôle crucial à jouer. Par exemple, l'Accord de Paris, signé en 2015, a pour objectif de limiter le réchauffement climatique à 1,5°C par rapport aux niveaux préindustriels. Cet accord international incite les nations à adopter des politiques ambitieuses pour réduire leurs émissions de CO2 et investir dans des technologies plus propres.

Cependant, la transition vers un modèle économique plus durable ne peut se faire sans des investissements significatifs dans la recherche et l'innovation. Les pays développés doivent également soutenir les nations en développement, qui sont souvent les plus vulnérables aux effets du changement climatique. La coopération internationale est donc essentielle pour garantir un avenir plus vert et plus juste pour tous.

La société civile a également un rôle à jouer. De plus en plus de personnes prennent conscience de l'impact de leurs actions quotidiennes sur l'environnement. La consommation responsable, la réduction des déchets, l'adoption de modes de transport durables sont autant d'exemples de changements positifs à l'échelle individuelle.

En conclusion, le changement climatique est un défi mondial qui nécessite une action immédiate et collective. Si nous voulons garantir un avenir viable pour les générations futures, il est impératif de mettre en œuvre des politiques de développement durable, de favoriser l'innovation et de sensibiliser chacun d'entre nous à l'importance de la préservation de notre planète."""


In [32]:
model.save_pretrained(r"C:\Users\Administrator\Desktop\dlrjrlaudwn\dlrj_rlaudwn\models\summarization_model\model.safetensors")
tokenizer.save_pretrained(r"C:\Users\Administrator\Desktop\dlrjrlaudwn\dlrj_rlaudwn\models\summarization_model\tokenizer_config.json")


Provided path (C:\Users\Administrator\Desktop\dlrjrlaudwn\dlrj_rlaudwn\models\summarization_model\model.safetensors) should be a directory, not a file
Provided path (C:\Users\Administrator\Desktop\dlrjrlaudwn\dlrj_rlaudwn\models\summarization_model\tokenizer_config.json) should be a directory, not a file


In [33]:
device = torch.device('cpu')  # CPU 장치로 설정

# 모델을 CPU로 이동
model = model.to(device)

# 입력 텐서를 CPU로 이동
input_ids = input_ids.to(device)

# 그 이후에 모델을 실행
output = model(input_ids)


[파라미터 설명]

-   max_length=300: 생성된 텍스트의 최대 길이를 300 토큰으로 제한합니다. 이는 전체 생성된 텍스트의 길이를 제한하는 것이므로, 3줄로 정확히 제한되지는 않습니다. 각 줄의 길이는 일반적으로 80~100자 정도로 추정할 수 있습니다.
-   max_new_tokens=150: 모델이 생성할 새로운 토큰의 최대 수를 150으로 제한합니다. 이 값은 기존 입력 텍스트를 포함하지 않고 모델이 새로 생성하는 텍스트의 길이를 제어합니다. 요약에서 중요한 부분은 max_new_tokens입니다.
-   num_beams=5: 빔 서치를 사용할 때 사용하는 빔의 수입니다. 빔 서치는 더 나은 결과를 찾기 위해 여러 후보 문장을 동시에 탐색하는 기법입니다.
-   early_stopping=True: 모델이 더 이상 유용한 정보를 생성하지 않을 때, 생성을 조기에 종료하도록 합니다. 이로 인해 불필요한 텍스트 생성을 방지할 수 있습니다.


In [36]:
# `attention_mask` 생성: 패딩된 토큰을 0으로 설정
attention_mask = (inputs['input_ids'] != tokenizer.pad_token_id).long()

# 모델과 토크나이저에서 pad_token_id를 동일하게 설정
tokenizer.pad_token_id = tokenizer.eos_token_id  # tokenizer의 pad_token_id를 eos_token_id로 설정
model.config.pad_token_id = tokenizer.pad_token_id  # 모델 설정에서 pad_token_id를 일치시킴

# 새로운 텍스트 요약하기
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_new_tokens=512,  num_beams=4, early_stopping=True,
                         no_repeat_ngram_size=2  # 반복 방지
                         )

# 생성된 요약 출력
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




Le Changement Climatique et le Développement Durable : Une Urgence Planétaire

Le changement climatique est l'un des plus grands défis auxquels l'humanité fait face au XXIe siècle. Les émissions de gaz à effet de serre, principalement dues à l'activité humaine, entraînent une hausse des températures mondiales, des phénomènes météorologiques extrêmes et une élévation du niveau des océans. Pour faire face à cette crise environnementale, le concept de développement durable est devenu essentiel.

Le développement durable repose sur l'idée qu'il est possible de répondre aux besoins des générations présentes sans compromettre la capacité des générations futures à satisfaire leurs propres besoins. Il s'agit de trouver un équilibre entre la protection de l'environnement, le progrès économique et l'amélioration des conditions sociales.

Les actions pour lutter contre le changement climatique incluent la réduction des émissions de gaz à effet de serre, la transition vers les énergies renouvela